# Supervised vs. unsupervised learning

Machine learning can be broadly divided into *supervised* and *unsupervised* tasks. In supervised learning, you have data where you know beforehand what the "correct" answer is. For example, in this portion of the workshop, we will develop a classifier to predict whether a given single-nucleotide variant is pathogenic or benign. For this task, we already have a set of thousands of variants where we know the proper answer for each. The classifier's task, then, is to learn what features of a variant are most strongly associated with pathogenic or benign status, allowing the classifier to make predictions about variants for which we don't know the correct response.

In unsupervised learning, by contrast, you have data for which you don't know what the correct answer is. This changes the types of questions you can ask about your data&mdash;creating a classifier for variant pathogenicity would be vastly more difficult if we only had a list of existing variants, with no knowledge of whether each variant was harmful. One example of unsupervised learning is clustering, where your group your data into sets sharing common features. This task is unsupervised because, before your clustering algorithm runs, you don't know how many clusters are in the data or what the features specific to each cluster are. In fact, different clustering algorithms will produce different answers, with one answer not necessarily more correct than another. Exactly which answer you prefer will depend on the purpose to which you will put the results.

Both supervised and unsupervised learning are critical elements of your machine learning toolbox.

# The problem

[ClinVar](https://www.ncbi.nlm.nih.gov/clinvar/) is a curated database of over 100,000 mutations in the human genome colelcted through research projects, clinical testing, or extraced from the literature by third parties. Each single-nucleotide variant involved in Mendelian disorders is characterized according to [criteria defined by the American College of Medical Genetics and Genomics](http://www.ncbi.nlm.nih.gov/pmc/articles/PMC4544753/), such that it falls into one of the following five classes:

* Pathogenic
* Likely pathogenic
* Likely benign
* Benign
* Unknown significance

The ACMG criteria for deciding a variant's class are listed below, with each factor assigned a weighting according to how likely human experts think it is in influencing pathogenicity. Note, for example, that a mutation inducing a nonsense mutation in a protein whose loss-of-function is known to cause disease has a "very strong" influence on pathogenicity, while a minor (mutant) allele frequency (MAF) amongst the human population substantially exceeding disease prevalence strongly suggests a variant is benign.

![ClinVar criteria](images/clinvar_criteria.jpg)

Our goal is simple: using ClinVar's mutation catalogue, we will train a classifier that can predict variant pathogenicity. You can imagine working on a project where you are trying to identify a *de novo* mutation leading to a Mendelian disorder, but you have observed thousands of candidate mutations amongst your afflicted cohort. Using our classifier trained on ClinVar, you can classify each of the thousands of mutations according to its probability of pathogenicity, vastly reducing the amount of data you must sift through to locate the disease cause.

To simplify our problem, we will perform only a binary classification task of deeming a given variant *pathogenic* or *benign*. Note, however, the methods presented here can be extended to multiclass problems, such that you could produce classifications across all five of the ClinVar categories.

# Feature selection

To decide whether a variant is benign or pathogenic, we need information about the variant. Each piece of information you have is deemed a *feature*. Your goal is to select the pieces of information most informative in making predictions, while reducing irrelevant features that only add noise. In machine learning, *feature selection* is a mundane but critical task&mdash;a simple algorithm using carefully selected features will often outperform a much more complex algorithm lacking access to good features.

For our ClinVar classifier, the simplest model would use only the immediately apparent information about the variant: the chromosome, position, reference allele, and alternate allele. For example, one variant could be represented simply as "chromosome 1, position 123,456, C to T mutation". This model would fare poorly, however, as far too little information is presented to permit a good guess as to pathogenicity&mdash;all of the surrounding genomic context is lost, with such sparse information meaning almost nothing in isolation. Though we could create a classifier using only these features, it would likely perform no better than chance.

Happily, ClinVar provides a great deal of metadata about each variant permitting much more accurate predictions as to pathogenicity. We will use the following features offered by ClinVar:

* *MAF*: minor allele frequency
* *ass*: variant in acceptor splice site
* *dss*: variant in donor splice site
* *int*: variant in intron
* *nsf*: induces frameshift
* *nsm*: induces missense
* *r3*: in 3' region of gene
* *r5*: in 5' region of gene
* *ref*: coding-region variant such that one allele in set is identical to reference
* *syn*: coding-region variant such that one allele in set is synonymous (i.e., does not change amino acid)
* *u3*: in 3' untranslated region
* *u5*: in 5' untranslated region

All features except *MAF* are simply boolean (true or false) values. Excepting *MAF*, every feature is defined for each variant in ClinVar, meaning that we don't need to impute values for missing data. Unfortunately, many real-world datasets do not provide values for each feature across all data, meaning that you must determine a sensible means of providing default values when such data are missing. In our case, we will resolve cases in which no MAF is provided for a variant by simply assigning to it the mean VAF observed across all variants.

# Logistic regression

# Preventing overfitting: training, test, and validation datasets

Now that we've created a logistic regression model to predict variant pathogenicity, we want to assess its performance. The simplest way of doing so would be to simply train the model on all variants in ClinVar, with the features for each variant and its true class (pathogenic or benign) provided. Then, to test performance, we feed each variant into the trained model once more, but without its label. We would then compare the model's prediction to the true label for the variant, seeing what proportion were correctly predicted.

The problem with so simple an approach is that your model will inevitably *overfit* your data. *Overfitting* means that your model is fit not to underlying patterns common to similar data sets&mdash;such as the combination of variant features most predictive of pathogenicity, like whether a mutation induces a missense mutation&mdash;but instead to noise particular to the data you have at hand. Overfitting often results when your model is trained with too little data, meaning it does not see sufficient data to generalize, or when your model is overly complex, with too many parameters that can be adjusted to precisely fit the peculiarities of your training data. 

![Overfitting example](images/overfitting_curves.png)
Consider the case of trying to fit polynomials of various orders to points generated from a sinusoidal curve (taken from Bishop). The green curve represents the underlying truth, with the blue points generated as points on the curve with the addition of some noise pushing them slightly away. We see that the zeroth- and first-degree polynomials, corresponding to $M=0$ and $M=1$, fit the points poorly. The third-order polynomial comes close to approximating the green curve, and so is deemed a good fit to the data. The ninth-order polynomial, however, overfits&mdash;though it passes precisely through every observed point, its structure is intrinsically tied to the noise that pushed the blue points off the green curve, and so it poorly represents the original curve. In this case, the model was too complex&mdash;with ten different parameters (i.e., coefficients for the ninth-order polynomial), the model was able to conform *too* closely to the observed data. Constraining model complexity such that we allow only a third-order polynomial, or providing the training process with more data, would have prevented such overfitting.
 
Now that we know what overfitting is, how do we prevent it? The key is to partition your data into *training* and *validation* sets. For example, you might train on 80% of your data, but then evaluate your model's performance on the remaining 20%. If your model overfits your training set, you will see model performance decrease on the validation set relative to a non-overfit model, allowing you to choose the model that best generalizes to inputs not seen during training.
 
But now we confront a different problem. You have only a limited amount of data, and you would like to evaluate your model against all available examples. You might imagine that, if you retain 20% of ClinVar variants to validate your model, that 20% is unusually easy or unusually hard to classify relative to the whole set, meaning that you will either overestimate or underestimate model perfromance, respectively. The solution is to use *k-fold cross validation*. Cross validation means that, once we partition the dataset into training and validation sets, we rotate through the partitions, changing which is used as the validation set each time. This way, we create three separate models from three distinct training sets and evaluate the performance on each, meaning that every variant will eventually be used to validate the model. This in turn gives us a better idea of how our model generalizes.

Cross-validation is particularly useful in selecting *hyperparameters*. While your model's *parameters* are automatically inferred from your training data&mdash;that's the *learning* part of *machine learning*&mdash;there are other parameters that can't be learned, and must be set by a human engineer. Such hyperparameters can have a significant effect on performance, as we will see when discussing regularization below. Typically, to select hyperparameters, we will repeat our entire k-fold cross validation scheme for each possible value of the parameter, selecting that which yields the best average performance on the validation set. This becomes difficult, however, when you have many hyperparameters. Suppose you have $n$ hyperparameters, each of which can take $k$ different values. This implies you must do a *grid search* over all possible $k^n$ combinations, repeating your entire training and validation procedure for each. This is an unfortunate problem to which there is no ready solution.

For our pathogenicity predictor, we will use three-fold cross validation. Each fold corresponds to a separate model we train. Observe how which portion of the data is used as validation and training changes in each fold.

| Fold | First 33% of data | Second 33% of data | Last 33% of data |                                                                    
| ---- | ----------------- | ------------------ | ----------------                                                                      
| First | Training | Training | Validation |                                                                                            
| Second | Training | Validation | Training |                                                                                           
| Third | Validation | Training | Training | 

Now, suppose we use our three-fold cross validation scheme to evaluate different models, choosing the one that best classifies our variants. Great! We're ready to publish. To maximize the amount of data our trained model has seen, we re-train our best model using all available variants in ClinVar, using the hyperparameters established as ideal during validation. The issue, however, is that you lack an unbiased means of estimating performance&mdash;it is critical that you perform your final evaluation of your model using data it has never seen before, to prevent precisely the overfitting issue we first encountered. To resolve this problem, before we ever begin to work on a model, we remove some of our data from our training/validation set and retain it as *test* data to use in the final evaluation. Suppose at the outset of this project you randomly select 20% of the ClinVar variants as your test set. Then, you proceed with training and evaluating different models with varying hyperparameters using the three-fold cross validation scheme we discussed on the remaining 80% of variants. Once you've selected the best model and hyperparameters, you re-train that model on all variants in your training/validation set. Finally, you evaluate this model's performance on the test set held out since starting your project. The most important consideration is that you cannot adjust your model or its hyperparameters to increase performance on the test set, as the test set should represent an entirely novel set of data your model has never before encountered, granting understanding into how well your model generalizes to real-world data for which the "correct" answer is unknown.

For our present project, we won't bother using a separate test data set, as we do not intend to use this model for real-world tasks. Nevertheless, using a separate test data set is essential when developing models for use in your own work.

# Assessing model performance

# Regularization

# Choosing hyperparameters

# Support vector machines

# Random forests